# Recommender Systems

### Learning Objectives:
- [Introduction: Simple Recommender Systems](#Introduction:-Simple-Recommeder-Systems)
- [Offline & Online Evaluation](#Offline-&-Online-Evaluation)
- [Content-based Recommenders](#Content\-based-Recommenders)
- [Collaborative-filtering](#Collaborative\-filtering)
- [Hybrid Systems](#Hybrid-Systems)


# Introduction: Simple Recommender Systems

__Recommender systems__, also referred to as __recommendation systems__, are filtering systems used by many different companies world-wide to be able to recommend products (e.g. movies, clothes, etc) based on user preferences. We obviously cannot recommend _exactly_ what a user wants as we cannot access or process all the information in their brain at the same time. Instead, we can take advantage or users'
past ratings and preferences to __predict__ the ratings of products users had not purchased/rated before and use these to estimate user preferences.

How do these systems do what they do? This is question that has become a large topic of research and the current answer is that there are mutliple ways to create recommender systems: each working under different assumptions and algorithms. There are two main broad classifications that we will cover shortly: __content-based recommendation__ and __collaborative filtering.__ 

Before we dive into these, we will cover what is informally referred to as a simple recommender system: a system that uses the weighted average rating from all users to make recommendations on the "best" options. Throughout this notebook, we will use the "Movies Dataset" from [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset), where the full version contains information on over 45,000 movies with 26 million ratings from  270,000 users. We will be using the small version, as shown below:

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [4]:
# Importing ratings
ratings = pd.read_csv("../DATA/ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
# Importing movie metadata
metadata = pd.read_csv("../DATA/movies_metadata.csv")
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


For our simple recommender system, we will use the IMDB's known __weighted average formula__ used for their Top Movies Chart, given as follows:

$$ R_{W} = (\frac{v}{v + m})R + (\frac{m}{v + m})C  $$

Where:
- $R_{W}$ is the weighted average movie rating
- $v$ is the number of votes for that movie title
- $m$ is the minimum number of votes required to be in the top Chart
- $R$ is the average rating of that movie title
- $C$ is the mean vote rating across all movies

We can now begin our calculations to construct our simple recommender:

In [8]:
# Computing mean vote count across all movies
vote_counts = metadata[metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = metadata[metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

We must now choose a value for the minimum number. In this case, we will choose a value $m$ that gives us movies that have received more votes than 95% of the other remaining movies.

In [9]:
# Computing minimum number of votes required
m = vote_counts.quantile(0.95)
print(m)

434.0


We can now extract the movies that are considered to be canditates for the top charts in our recommender system given our computed 'm'.

In [15]:
# Extracting all movies that have a votecount that is greater than our m value
qualified = metadata[(metadata['vote_count'] >= m) & (metadata['vote_count'].notnull()) & (metadata['vote_average'].notnull())][['title', 'release_date', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [17]:
# Computing weighted average
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['weighted_average'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('weighted_average', ascending=False).head(250)
qualified.head(15)

,title,release_date,vote_count,vote_average,popularity,genres,weighted_average
15480,Inception,2010-07-14,14075,8,29.1081,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",7.917588
12481,The Dark Knight,2008-07-16,12269,8,123.167,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",7.905871
22879,Interstellar,2014-11-05,11187,8,32.2135,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",7.897107
2843,Fight Club,1999-10-15,9678,8,63.8696,"[{'id': 18, 'name': 'Drama'}]",7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,8892,8,32.0707,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.871787
292,Pulp Fiction,1994-09-10,8670,8,140.95,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",7.868660
314,The Shawshank Redemption,1994-09-23,8358,8,51.6454,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.864000
7000,The Lord of the Rings: The Return of the King,2003-12-01,8226,8,29.3244,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.861927
351,Forrest Gump,1994-07-06,8147,8,48.3072,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",7.860656
5814,The Lord of the Rings: The Two Towers,2002-12-18,7641,8,29.4235,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.851924


# Offline & Online Evaluation

# Content-based Recommenders

# Collaborative-filtering

# Hybrid Systems